In [16]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [6]:
df = pd.read_csv('generations.csv')
df.head()

,Unnamed: 0,size,document,splitter,score,chunk,question,relevance_rating,groundness_rating,standalone_rating,limit,collection,context,ctx_length,answer,model
0,0,1000,insurance_act,char,0.8,"the Special Economic Zones Act, 2005 (28 of 20...",What legal consequences (penalties) exist for ...,5,5,5,3,semantic,['26-12-2014). 40B. Limitation of expenses of ...,23940,Individuals or entities who violate the provis...,meta-llama-3.1-8b-instruct
1,1,1000,insurance_act,char,0.8,"the Special Economic Zones Act, 2005 (28 of 20...",What legal consequences (penalties) exist for ...,5,5,5,3,char,['domiciled outside India or any insurer as de...,5844,Individuals or entities who violate the provis...,meta-llama-3.1-8b-instruct
2,2,1000,insurance_act,char,0.8,"the Special Economic Zones Act, 2005 (28 of 20...",What legal consequences (penalties) exist for ...,5,5,5,3,token,"['thereof shall, in all legal proceedings (whe...",27040,Individuals or entities who violate the provis...,meta-llama-3.1-8b-instruct
3,3,1000,insurance_act,char,0.8,"the Special Economic Zones Act, 2005 (28 of 20...",What legal consequences (penalties) exist for ...,5,5,5,3,recursive,['(i) “policy” means a policy issued or renewe...,26431,"According to the context, individuals or entit...",meta-llama-3.1-8b-instruct
4,4,1000,insurance_act,char,0.8,"the Special Economic Zones Act, 2005 (28 of 20...",What legal consequences (penalties) exist for ...,5,5,5,3,semantic,['26-12-2014). 40B. Limitation of expenses of ...,23940,Individuals or entities violating provisions o...,meta-llama-3.1-8b-instruct


In [9]:
df['size'].unique()

array(['1000', '2000', '3000', 'insurance_act', 'policyholder',
       'handbook'], dtype=object)

In [12]:
formatted_df = pd.DataFrame(columns=df.columns)
unformmated_df = pd.DataFrame(columns=df.columns)

for idx, row in tqdm(df.iterrows()):
    if row['size'].isnumeric():
        formatted_df.loc[len(formatted_df)] = row
    else:
        unformmated_df.loc[len(unformmated_df)] = row

3566it [00:02, 1209.67it/s]


In [17]:
shifted_array = np.empty_like(unformmated_df.values, dtype=object)
shifted_array[:, 0] = np.nan
shifted_array[:, 1:] = unformmated_df.values[:, :-1]
shifted_df = pd.DataFrame(shifted_array, columns=df.columns)

In [22]:
formatted_df.tail()

,Unnamed: 0,size,document,splitter,score,chunk,question,relevance_rating,groundness_rating,standalone_rating,limit,collection,context,ctx_length,answer,model
3471,3480,3000,handbook,char,0.85,stocks with the general aim of \ncapital appre...,What specific types of fixed income instrument...,5,5,5,3,optimized-recursive,['roved securities” means—\n(i) Government sec...,4152,('The types of fixed income instruments invest...,meta-llama-3.1-8b-instruct
3472,3481,3000,handbook,char,0.85,stocks with the general aim of \ncapital appre...,What specific types of fixed income instrument...,5,5,5,5,semantic,['Surrender value is usually expressed as fund...,29460,"('In Unit Linked Policies, the types of fixed ...",meta-llama-3.1-8b-instruct
3473,3482,3000,handbook,char,0.85,stocks with the general aim of \ncapital appre...,What specific types of fixed income instrument...,5,5,5,5,char,['life insurance or any other class or sub-cla...,9804,"('Based on the context provided, Unit Linked P...",meta-llama-3.1-8b-instruct
3474,3483,3000,handbook,char,0.85,stocks with the general aim of \ncapital appre...,What specific types of fixed income instrument...,5,5,5,5,token,"['THE INSURANCE ACT, 1938\nACT NO. 4 OF 19381\...",39090,('The types of fixed income instruments invest...,meta-llama-3.1-8b-instruct
3475,3484,3000,handbook,char,0.85,stocks with the general aim of \ncapital appre...,What specific types of fixed income instrument...,5,5,5,5,recursive,"['THE INSURANCE ACT, 1938\nACT NO. 4 OF 19381\...",38266,"(""Corporate bonds invest in fixed income instr...",meta-llama-3.1-8b-instruct


In [28]:
formatted_df.drop(columns=['Unnamed: 0'], inplace=True)
shifted_df.drop(columns=['Unnamed: 0'], inplace=True)

In [35]:
def clean_string(answer):
    try:
        return eval(answer)[0]
    except:
        return answer

In [36]:
formatted_df['answer'] = formatted_df['answer'].apply(clean_string)
shifted_df['answer'] = shifted_df['answer'].apply(clean_string)

In [42]:
formatted_df.shape, shifted_df.shape

((3476, 15), (90, 15))

In [43]:
df = pd.concat([formatted_df, shifted_df], axis=0)

In [45]:
df = df[df['answer'].str.len() > 10]

In [46]:
df.shape

(2975, 15)

In [47]:
df['model'].value_counts()

model
meta-llama-3.1-8b-instruct    1792
gemma-3-1b-it                 1183
Name: count, dtype: int64

In [48]:
df['collection'].value_counts()

collection
char                   394
optimized-token        388
semantic               376
optimized-char         370
optimized-semantic     363
token                  363
optimized-recursive    361
recursive              360
Name: count, dtype: int64

In [49]:
df.to_csv('generations.csv', index=False)

In [10]:
questions = pd.read_csv('Eval\question-evals.csv')

<>:1: SyntaxWarning: invalid escape sequence '\q'
<>:1: SyntaxWarning: invalid escape sequence '\q'
C:\Users\shour\AppData\Local\Temp\ipykernel_28916\1607386637.py:1: SyntaxWarning: invalid escape sequence '\q'
  questions = pd.read_csv('Eval\question-evals.csv')


In [12]:
questions = questions[
    (questions['relevance_rating'] == 5) &
    (questions['groundness_rating'] == 5) &
    (questions['standalone_rating'] == 5)
]

In [13]:
questions.shape

(116, 10)